In [1]:
#Document qui vise à explorer le dataset de la base de données de KBO

#Importation des librairies
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

In [2]:
# Chemin relatif vers le dossier de données
data_dir = os.path.join('..', '..', 'data')

# Chemins complets vers chaque fichier CSV
activity_path = os.path.join(data_dir, 'activity.csv')
address_path = os.path.join(data_dir, 'address.csv')
#branch_path = os.path.join(data_dir, 'branch.csv')
#code_path = os.path.join(data_dir, 'code.csv')
#contact_path = os.path.join(data_dir, 'contact.csv')
#denomination_path = os.path.join(data_dir, 'denomination.csv')
enterprise_path = os.path.join(data_dir, 'enterprise.csv')
#establishment_path = os.path.join(data_dir, 'establishment.csv')
#meta_path = os.path.join(data_dir, 'meta.csv')

In [3]:
# Chargement des datasets
activity_df = pd.read_csv(activity_path)
address_df = pd.read_csv(address_path)
#branch_df = pd.read_csv(branch_path)
#code_df = pd.read_csv(code_path)
#contact_df = pd.read_csv(contact_path)
#denomination_df = pd.read_csv(denomination_path)
enterprise_df = pd.read_csv(enterprise_path)
#establishment_df = pd.read_csv(establishment_path)
#meta_df = pd.read_csv(meta_path)

C:\Users\Nawfel\AppData\Local\Temp\ipykernel_23828\1763326358.py:3: DtypeWarning: Columns (2,3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  address_df = pd.read_csv(address_path)


In [4]:
# Afficher les premières lignes de chaque dataset pour vérifier qu'ils sont chargés correctement
print("Activity Dataset:")
print(activity_df.head())

print("\nAddress Dataset:")
print(address_df.head())

#print("\nBranch Dataset:")
#print(branch_df.head())

#print("\nCode Dataset:")
#print(code_df.head())

#print("\nContact Dataset:")
#print(contact_df.head())

#print("\nDenomination Dataset:")
#print(denomination_df.head())

print("\nEnterprise Dataset:")
print(enterprise_df.head())

#print("\nEstablishment Dataset:")
#print(establishment_df.head())

#print("\nMeta Dataset:")
#print(meta_df.head())

Activity Dataset:
   EntityNumber  ActivityGroup  NaceVersion  NaceCode Classification
0  0200.065.765              6         2008     84130           MAIN
1  0200.065.765              1         2008     41101           MAIN
2  0200.065.765              1         2003     70111           MAIN
3  0200.068.636              6         2008     36000           MAIN
4  0200.068.636              1         2008     36000           MAIN

Address Dataset:
   EntityNumber TypeOfAddress CountryNL CountryFR Zipcode  \
0  0200.065.765          REGO       NaN       NaN    9070   
1  0200.068.636          REGO       NaN       NaN    9000   
2  0200.171.970          REGO       NaN       NaN    9000   
3  0200.245.711          REGO       NaN       NaN    9500   
4  0200.305.493          REGO       NaN       NaN    9520   

        MunicipalityNL       MunicipalityFR        StreetNL        StreetFR  \
0         Destelbergen         Destelbergen   Panhuisstraat   Panhuisstraat   
1                 Gent   

In [5]:
def optimize_dataframe(df):
    for col in df.columns:
        col_type = df[col].dtype
        if col_type == 'float64':
            df[col] = df[col].astype('float32')
        elif col_type == 'int64':
            df[col] = df[col].astype('int32')
        elif col_type == 'object':
            num_unique_values = len(df[col].unique())
            num_total_values = len(df[col])
            if num_unique_values / num_total_values < 0.5:
                df[col] = df[col].astype('category')
    return df

# Optimiser les datasets
activity_df = optimize_dataframe(activity_df)
address_df = optimize_dataframe(address_df)
enterprise_df = optimize_dataframe(enterprise_df)

In [6]:
# Fusionner les datasets en utilisant EntityNumber comme clé primaire
merged_df = activity_df.merge(address_df, on='EntityNumber', how='left')
merged_df = merged_df.merge(enterprise_df, on='EntityNumber', how='left')


KeyError: 'EntityNumber'

In [ ]:
# Préparer les features et la target
features = merged_df[['ActivityGroup', 'NaceVersion', 'NaceCode', 'Zipcode', 'JuridicalSituation', 'TypeOfEnterprise', 'JuridicalForm']]
target = merged_df['Classification'].apply(lambda x: 1 if x == 'MAIN' else 0)  # Exemple de binaire

In [ ]:

# Remplacer les valeurs manquantes par la moyenne ou la valeur la plus fréquente
features = features.fillna(features.mean())

In [ ]:
# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Entraîner le modèle de régression linéaire
model = LinearRegression()
model.fit(X_train, y_train)

# Prédire sur l'ensemble de test
y_pred = model.predict(X_test)

# Évaluer le modèle
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')